# Data Exploration: Next Steps (Polars-only, Scalable)

This notebook assumes that you have already run **`data_exploration_organized.ipynb`**.

From that notebook, the following outputs are expected in `data/results`:

- Base tabular features: `X_train.parquet`, `X_val.parquet`, `X_test.parquet`
- Labels (if supervised): `y_train.npy`, `y_val.npy`
- NLP embedding features, per model and split, for example:
  - `sent_transformer_X_train.parquet`, `sent_transformer_X_val.parquet`, `sent_transformer_X_test.parquet`
  - `scibert_X_train.parquet`, `scibert_X_val.parquet`, `scibert_X_test.parquet`
  - (optionally) `specter2_X_train.parquet`, `specter2_X_val.parquet`, `specter2_X_test.parquet`

Each embedding parquet contains an `id` column plus one column per embedding dimension
(e.g. `sent_transformer_dim_0`, `sent_transformer_dim_1`, ...).

This notebook will:

1. Load the **base feature matrices** and all available **embedding parquets**
2. Merge embeddings into the base feature matrices by `id`
3. Inspect and impute missing values in a train-centric way
4. Replace `is_oa` with derived feature `is_not_oa` (1 - is_oa) for better signal
5. Run a set of statistical analyses (Pearson, Spearman, Chi-square, Cramér's V, ANOVA, Tukey's HSD)
6. Keep **all non-embedding features** (no feature reduction) to preserve signals that may be sparse in sample but informative in full dataset
7. Save model-ready train / validation / test parquet files for downstream ML

Implementation is **Polars-only** for dataframes (no pandas). NumPy, SciPy and statsmodels are used only for
vectorised numerical routines.

## 1. Imports & directory configuration

In [1]:
import os
from pathlib import Path
from typing import Dict, List, Any

import numpy as np
import polars as pl

from scipy import stats

# Optional: statsmodels for Tukey's HSD
try:
    import statsmodels.api as sm
    from statsmodels.stats.multicomp import pairwise_tukeyhsd
    HAS_STATSMODELS = True
except ImportError:
    HAS_STATSMODELS = False
    print("⚠️ statsmodels not found. Tukey's HSD will be skipped unless you install statsmodels.")

_current_dir = Path(os.getcwd()).parent.parent
if (_current_dir / "data").exists():
    PROJECT_ROOT = _current_dir
else:
    PROJECT_ROOT = Path.cwd()

DATA_DIR    = PROJECT_ROOT / "data/processed"
RESULTS_DIR = PROJECT_ROOT / "data/results"
MODEL_DIR   = PROJECT_ROOT / "data/model_ready"
MODEL_DIR.mkdir(parents=True, exist_ok=True)

print(f"PROJECT_ROOT: {PROJECT_ROOT}")
print(f"DATA_DIR:     {DATA_DIR}")
print(f"RESULTS_DIR:  {RESULTS_DIR}")
print(f"MODEL_DIR:    {MODEL_DIR}")

PROJECT_ROOT: /gpfs/accounts/si670f25_class_root/si670f25_class/santoshd/Kaggle_2
DATA_DIR:     /gpfs/accounts/si670f25_class_root/si670f25_class/santoshd/Kaggle_2/data/processed
RESULTS_DIR:  /gpfs/accounts/si670f25_class_root/si670f25_class/santoshd/Kaggle_2/data/results
MODEL_DIR:    /gpfs/accounts/si670f25_class_root/si670f25_class/santoshd/Kaggle_2/data/model_ready


## 2. Load base feature matrices & labels

In [2]:
X_train_path = RESULTS_DIR / "X_train.parquet"
X_val_path   = RESULTS_DIR / "X_val.parquet"
X_test_path  = RESULTS_DIR / "X_test.parquet"

print("Base feature file locations:")
print("  ", X_train_path)
print("  ", X_val_path)
print("  ", X_test_path)

if not X_train_path.exists() or not X_val_path.exists() or not X_test_path.exists():
    raise FileNotFoundError(
        "One or more X_*.parquet files are missing in data/results. "
        "Please run data_exploration_organized.ipynb first."
    )

X_train_base = pl.read_parquet(X_train_path)
X_val_base   = pl.read_parquet(X_val_path)
X_test_base  = pl.read_parquet(X_test_path)

print("\nBase shapes (before merging embeddings):")
print("  X_train_base:", X_train_base.shape)
print("  X_val_base:  ", X_val_base.shape)
print("  X_test_base: ", X_test_base.shape)

y_train_path = RESULTS_DIR / "y_train.npy"
y_val_path   = RESULTS_DIR / "y_val.npy"

y_train = np.load(y_train_path) if y_train_path.exists() else None
y_val   = np.load(y_val_path)   if y_val_path.exists()   else None

if y_train is not None:
    print("\nLoaded labels:")
    print("  y_train:", y_train.shape)
    if y_val is not None:
        print("  y_val:  ", y_val.shape)
else:
    print("\nNo y_train.npy found. Target-based analyses will be limited.")

Base feature file locations:
   /gpfs/accounts/si670f25_class_root/si670f25_class/santoshd/Kaggle_2/data/results/X_train.parquet
   /gpfs/accounts/si670f25_class_root/si670f25_class/santoshd/Kaggle_2/data/results/X_val.parquet
   /gpfs/accounts/si670f25_class_root/si670f25_class/santoshd/Kaggle_2/data/results/X_test.parquet



Base shapes (before merging embeddings):
  X_train_base: (960000, 57)
  X_val_base:   (120000, 57)
  X_test_base:  (120000, 57)



Loaded labels:
  y_train: (960000,)
  y_val:   (120000,)


## 3. Discover and merge embedding parquets by `id`

In [3]:
def discover_embedding_models(results_dir: Path) -> List[str]:
    models: List[str] = []
    for p in results_dir.glob("*_X_train.parquet"):
        name = p.name.replace("_X_train.parquet", "")
        if name in {"X", "base", "features"}:
            continue
        models.append(name)
    return sorted(set(models))

embedding_models = discover_embedding_models(RESULTS_DIR)
print("Discovered embedding models (based on *_X_train.parquet):")
print(embedding_models if embedding_models else "(none)")

def load_embedding_split(model_name: str, split: str) -> pl.DataFrame:
    path = RESULTS_DIR / f"{model_name}_X_{split}.parquet"
    if not path.exists():
        raise FileNotFoundError(f"Expected embedding file not found: {path}")
    df = pl.read_parquet(path)
    if "id" not in df.columns:
        raise ValueError(f"Embedding file {path} does not contain an 'id' column.")
    cols = ["id"] + [c for c in df.columns if c != "id"]
    return df.select(cols)

def merge_embeddings(
    X_train_base: pl.DataFrame,
    X_val_base: pl.DataFrame,
    X_test_base: pl.DataFrame,
    models: List[str],
) -> tuple[pl.DataFrame, pl.DataFrame, pl.DataFrame]:
    X_train = X_train_base
    X_val   = X_val_base
    X_test  = X_test_base

    if "id" not in X_train.columns:
        raise ValueError("Base feature matrices must contain an 'id' column for merging embeddings.")

    for model_name in models:
        # Check if all required files exist for this model
        required_files = [
            RESULTS_DIR / f"{model_name}_X_train.parquet",
            RESULTS_DIR / f"{model_name}_X_val.parquet",
            RESULTS_DIR / f"{model_name}_X_test.parquet"
        ]
        missing_files = [f for f in required_files if not f.exists()]
        
        if missing_files:
            print(f"\n⚠️  Skipping {model_name} embeddings - missing files:")
            for f in missing_files:
                print(f"     {f.name}")
            continue
        
        print(f"\n🔗 Merging embeddings for model: {model_name}")
        for split in ["train", "val", "test"]:
            try:
                emb_df = load_embedding_split(model_name, split)
                if split == "train":
                    before_cols = len(X_train.columns)
                    X_train = X_train.join(emb_df, on="id", how="left")
                    after_cols = len(X_train.columns)
                    print(f"  train: {before_cols} -> {after_cols} columns after merging {model_name}")
                elif split == "val":
                    before_cols = len(X_val.columns)
                    X_val = X_val.join(emb_df, on="id", how="left")
                    after_cols = len(X_val.columns)
                    print(f"  val:   {before_cols} -> {after_cols} columns after merging {model_name}")
                else:
                    before_cols = len(X_test.columns)
                    X_test = X_test.join(emb_df, on="id", how="left")
                    after_cols = len(X_test.columns)
                    print(f"  test:  {before_cols} -> {after_cols} columns after merging {model_name}")
            except (FileNotFoundError, ValueError) as e:
                print(f"  ⚠️  Skipping {split} split for {model_name}: {e}")
                continue

    return X_train, X_val, X_test

if embedding_models:
    X_train, X_val, X_test = merge_embeddings(X_train_base, X_val_base, X_test_base, embedding_models)
else:
    print("\nNo embedding parquets discovered; using base features only.")
    X_train, X_val, X_test = X_train_base, X_val_base, X_test_base

print("\nShapes AFTER embedding merge:")
print("  X_train:", X_train.shape)
print("  X_val:  ", X_val.shape)
print("  X_test: ", X_test.shape)

Discovered embedding models (based on *_X_train.parquet):
['scibert', 'sent_transformer', 'specter2']

🔗 Merging embeddings for model: scibert


  train: 57 -> 825 columns after merging scibert


  val:   57 -> 825 columns after merging scibert


  test:  57 -> 825 columns after merging scibert

🔗 Merging embeddings for model: sent_transformer


  train: 825 -> 1209 columns after merging sent_transformer


  val:   825 -> 1209 columns after merging sent_transformer


  test:  825 -> 1209 columns after merging sent_transformer

🔗 Merging embeddings for model: specter2


  train: 1209 -> 1977 columns after merging specter2


  val:   1209 -> 1977 columns after merging specter2


  test:  1209 -> 1977 columns after merging specter2

Shapes AFTER embedding merge:
  X_train: (960000, 1977)
  X_val:   (120000, 1977)
  X_test:  (120000, 1977)


## 4. Dataset overview & missingness diagnostics

In [4]:
def describe_dataframe(df: pl.DataFrame, name: str) -> None:
    print(f"\n{'='*80}")
    print(f"Overview: {name}")
    print(f"{'='*80}")
    print("Shape:", df.shape)
    print("\nDtypes:")
    print(df.dtypes)
    print("\nHead:")
    print(df.head())

describe_dataframe(X_train, "X_train (with embeddings)")
describe_dataframe(X_val,   "X_val (with embeddings)")
describe_dataframe(X_test,  "X_test (with embeddings)")

train_nulls = X_train.select(pl.all().null_count()).to_dicts()[0]
val_nulls   = X_val.select(pl.all().null_count()).to_dicts()[0]
test_nulls  = X_test.select(pl.all().null_count()).to_dicts()[0]

rows_train = X_train.height
rows_val   = X_val.height
rows_test  = X_test.height

records = []
for col in X_train.columns:
    records.append({
        "column": col,
        "train_missing": train_nulls.get(col, 0),
        "val_missing":   val_nulls.get(col, 0),
        "test_missing":  test_nulls.get(col, 0),
        "train_pct": train_nulls.get(col, 0) / rows_train if rows_train > 0 else 0.0,
        "val_pct":   val_nulls.get(col, 0) / rows_val   if rows_val > 0 else 0.0,
        "test_pct":  test_nulls.get(col, 0) / rows_test  if rows_test > 0 else 0.0,
    })

missing_report = pl.DataFrame(records).sort("train_missing", descending=True)
print("\nColumns with most missing values (train perspective):")
print(missing_report.head(30))

missing_report_path = MODEL_DIR / "missingness_report.csv"
missing_report.write_csv(missing_report_path)
print(f"\n💾 Missingness report saved to: {missing_report_path}")


Overview: X_train (with embeddings)
Shape: (960000, 1977)

Dtypes:
[Int64, Int64, Float64, Float64, Float64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, String, Int64, Int64, Int64, String, Int64, Float64, Int64, Float64, Float64, Float64, Int64, Float64, Float64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, Int64, String, Int64, Int64, Int64, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32

shape: (5, 1_977)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ abstract_ ┆ abstract_ ┆ avg_autho ┆ avg_autho ┆ … ┆ specter2_ ┆ specter2_ ┆ specter2_ ┆ specter2 │
│ length    ┆ word_coun ┆ r_citatio ┆ r_h_index ┆   ┆ dim_96    ┆ dim_97    ┆ dim_98    ┆ _dim_99  │
│ ---       ┆ t         ┆ ns        ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ i64       ┆ ---       ┆ ---       ┆ f64       ┆   ┆ f32       ┆ f32       ┆ f32       ┆ f32      │
│           ┆ i64       ┆ f64       ┆           ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 992       ┆ 138       ┆ 0.0       ┆ 0.0       ┆ … ┆ -0.900445 ┆ 0.003919  ┆ 0.136321  ┆ 0.725182 │
│ 0         ┆ 0         ┆ 0.0       ┆ 0.0       ┆ … ┆ -0.463011 ┆ -0.412438 ┆ -0.185211 ┆ 0.617742 │
│ 3101      ┆ 455       ┆ 0.0       ┆ 0.0       ┆ … ┆ -0.517267 ┆ -0.2627


Columns with most missing values (train perspective):
shape: (30, 7)
┌───────────────────┬───────────────┬─────────────┬──────────────┬───────────┬──────────┬──────────┐
│ column            ┆ train_missing ┆ val_missing ┆ test_missing ┆ train_pct ┆ val_pct  ┆ test_pct │
│ ---               ┆ ---           ┆ ---         ┆ ---          ┆ ---       ┆ ---      ┆ ---      │
│ str               ┆ i64           ┆ i64         ┆ i64          ┆ f64       ┆ f64      ┆ f64      │
╞═══════════════════╪═══════════════╪═════════════╪══════════════╪═══════════╪══════════╪══════════╡
│ oa_status_bronze  ┆ 896574        ┆ 112319      ┆ 112234       ┆ 0.933931  ┆ 0.935992 ┆ 0.935283 │
│ oa_status_hybrid  ┆ 894459        ┆ 112332      ┆ 112282       ┆ 0.931728  ┆ 0.9361   ┆ 0.935683 │
│ oa_status_green   ┆ 884087        ┆ 110609      ┆ 110525       ┆ 0.920924  ┆ 0.921742 ┆ 0.921042 │
│ oa_status_diamond ┆ 855879        ┆ 107382      ┆ 99511        ┆ 0.891541  ┆ 0.89485  ┆ 0.829258 │
│ oa_status_gold    ┆


💾 Missingness report saved to: /gpfs/accounts/si670f25_class_root/si670f25_class/santoshd/Kaggle_2/data/model_ready/missingness_report.csv


## 5. Train-centric missing value imputation (including embeddings)

In [5]:
id_cols = [c for c in X_train.columns if c.lower() == "id"]
feature_cols = [c for c in X_train.columns if c not in id_cols]

train_nulls = X_train.select(pl.all().null_count()).to_dicts()[0]
all_missing_cols = [c for c in feature_cols if train_nulls.get(c, 0) == X_train.height]

if all_missing_cols:
    print("Dropping columns that are entirely missing in train:")
    print(all_missing_cols)
    X_train = X_train.drop(all_missing_cols)
    X_val   = X_val.drop([c for c in all_missing_cols if c in X_val.columns])
    X_test  = X_test.drop([c for c in all_missing_cols if c in X_test.columns])
    feature_cols = [c for c in feature_cols if c not in all_missing_cols]

embedding_cols: List[str] = []
for m in embedding_models:
    prefix = f"{m}_"
    embedding_cols.extend([c for c in feature_cols if c.startswith(prefix)])
embedding_cols = sorted(set(embedding_cols))

numeric_cols: List[str] = []
categorical_cols: List[str] = []

NUMERIC_DTYPES = {
    pl.Int8, pl.Int16, pl.Int32, pl.Int64,
    pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64,
    pl.Float32, pl.Float64
}

for c, dt in zip(X_train.columns, X_train.dtypes):
    if c in feature_cols:
        if dt in NUMERIC_DTYPES:
            numeric_cols.append(c)
        else:
            categorical_cols.append(c)

print("\n# Features by type (after dropping all-missing columns)")
print("Numeric features    :", len(numeric_cols))
print("Categorical features:", len(categorical_cols))
print("Embedding features  :", len(embedding_cols))

numeric_exprs = [pl.col(c).median().alias(c) for c in numeric_cols]
numeric_medians_row = X_train.select(numeric_exprs).to_dicts()[0] if numeric_exprs else {}
numeric_impute_values: Dict[str, float] = {c: float(v) for c, v in numeric_medians_row.items()}

categorical_impute_values: Dict[str, Any] = {}
for col in categorical_cols:
    mode_df = X_train.select(pl.col(col).mode())
    if mode_df.height > 0:
        mode_val = mode_df.select(pl.col(col))[0, 0]
        categorical_impute_values[col] = mode_val
    else:
        categorical_impute_values[col] = "__missing__"

print("\nSample numeric impute values:")
for k in list(numeric_impute_values.keys())[:10]:
    print(f"  {k}: {numeric_impute_values[k]}")

print("\nSample categorical impute values:")
for k in list(categorical_impute_values.keys())[:10]:
    print(f"  {k}: {categorical_impute_values[k]}")

def apply_imputation(
    df: pl.DataFrame,
    numeric_values: Dict[str, float],
    categorical_values: Dict[str, Any],
    id_cols: List[str],
) -> pl.DataFrame:
    out = df

    for col, val in numeric_values.items():
        if col in out.columns:
            out = out.with_columns(pl.col(col).cast(pl.Float64).fill_null(val))

    for col, val in categorical_values.items():
        if col in out.columns:
            out = out.with_columns(pl.col(col).cast(pl.Utf8).fill_null(str(val)))

    remaining_nulls = out.select(pl.all().null_count()).to_dicts()[0]
    remaining_cols = [c for c, cnt in remaining_nulls.items() if cnt > 0]
    if remaining_cols:
        print("\nColumns still with missing values after primary imputation:", remaining_cols)
        for c in remaining_cols:
            if c in numeric_values:
                out = out.with_columns(pl.col(c).fill_null(0.0))
            else:
                out = out.with_columns(pl.col(c).cast(pl.Utf8).fill_null("__missing__"))

    ordered_cols = id_cols + [c for c in out.columns if c not in id_cols]
    return out.select(ordered_cols)

X_train_imputed = apply_imputation(X_train, numeric_impute_values, categorical_impute_values, id_cols)
X_val_imputed   = apply_imputation(X_val,   numeric_impute_values, categorical_impute_values, id_cols)
X_test_imputed  = apply_imputation(X_test,  numeric_impute_values, categorical_impute_values, id_cols)

print("\nPost-imputation missingness (train):")
print(X_train_imputed.select(pl.all().null_count()).transpose())

import json
impute_spec = {
    "numeric_impute_values": numeric_impute_values,
    "categorical_impute_values": {k: str(v) for k, v in categorical_impute_values.items()},
    "dropped_all_missing_cols": all_missing_cols,
    "id_cols": id_cols,
    "embedding_models": embedding_models,
    "embedding_cols": embedding_cols,
}
impute_spec_path = MODEL_DIR / "imputation_spec.json"
with impute_spec_path.open("w") as f:
    json.dump(impute_spec, f, indent=2)
print(f"\n💾 Imputation spec saved to: {impute_spec_path}")

# Replace is_oa with is_not_oa (derived feature: 1 - is_oa)
# This provides better signal for class imbalance scenarios
if "is_oa" in X_train_imputed.columns:
    print("\n🔄 Replacing is_oa with is_not_oa (derived feature)...")
    X_train_imputed = X_train_imputed.with_columns(
        (1 - pl.col("is_oa")).alias("is_not_oa")
    ).drop("is_oa")
    X_val_imputed = X_val_imputed.with_columns(
        (1 - pl.col("is_oa")).alias("is_not_oa")
    ).drop("is_oa")
    X_test_imputed = X_test_imputed.with_columns(
        (1 - pl.col("is_oa")).alias("is_not_oa")
    ).drop("is_oa")
    print("✅ Replaced is_oa with is_not_oa in all datasets")
else:
    print("\n⚠️  is_oa column not found, skipping replacement")



# Features by type (after dropping all-missing columns)
Numeric features    : 1974
Categorical features: 2
Embedding features  : 1920



Sample numeric impute values:
  abstract_length: 219.0
  abstract_word_count: 31.0
  avg_author_citations: 0.0
  avg_author_h_index: 0.0
  avg_concept_score: 0.34736020060606065
  avg_topic_score: 0.0
  first_author_citations: 0.0
  first_author_h_index: 0.0
  first_author_papers: 0.0
  has_abstract: 1.0

Sample categorical impute values:
  language: en
  type: article



Post-imputation missingness (train):
shape: (1_977, 1)
┌──────────┐
│ column_0 │
│ ---      │
│ u32      │
╞══════════╡
│ 0        │
│ 0        │
│ 0        │
│ 0        │
│ 0        │
│ …        │
│ 0        │
│ 0        │
│ 0        │
│ 0        │
│ 0        │
└──────────┘

💾 Imputation spec saved to: /gpfs/accounts/si670f25_class_root/si670f25_class/santoshd/Kaggle_2/data/model_ready/imputation_spec.json

🔄 Replacing is_oa with is_not_oa (derived feature)...
✅ Replaced is_oa with is_not_oa in all datasets


## 6. Statistical analyses (correlations, chi-square, Cramér's V, ANOVA, Tukey's HSD)

In [6]:
if y_train is None:
    print("No y_train.npy found. Skipping label-based statistical analyses.")

else:
    label_series = y_train
    uniq = np.unique(label_series)
    n_unique = len(uniq)
    is_label_categorical = (n_unique <= 20)

    print("\nLabel diagnostics:")
    print("  dtype        :", label_series.dtype)
    print("  # unique vals:", n_unique)
    print("  treated as categorical:", is_label_categorical)

    print("\n=== 6.1 Pearson & Spearman correlations ===")
    numeric_feature_cols = [
        c for c, dt in zip(X_train_imputed.columns, X_train_imputed.dtypes)
        if dt in {
            pl.Int8, pl.Int16, pl.Int32, pl.Int64,
            pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64,
            pl.Float32, pl.Float64
        }
    ]

    if np.issubdtype(label_series.dtype, np.number):
        label_numeric = label_series.astype(float)
    else:
        _, label_numeric = np.unique(label_series, return_inverse=True)
        label_numeric = label_numeric.astype(float)
        print("Label converted to numeric codes for correlation analysis.")

    corr_records: List[Dict[str, Any]] = []
    for col in numeric_feature_cols:
        x = X_train_imputed.select(pl.col(col)).to_series().to_numpy().astype(float)
        mask = ~np.isnan(x) & ~np.isnan(label_numeric)
        if mask.sum() < 10:
            continue

        x_vals = x[mask]
        y_vals = label_numeric[mask]

        # Skip if either array is constant (all values the same)
        if len(np.unique(x_vals)) <= 1 or len(np.unique(y_vals)) <= 1:
            continue

        try:
            pearson_r, pearson_p = stats.pearsonr(x_vals, y_vals)
            spearman_r, spearman_p = stats.spearmanr(x_vals, y_vals)

            # Handle NaN results
            if np.isnan(pearson_r) or np.isnan(spearman_r):
                continue
        except Exception:
            continue

        corr_records.append({
            "feature": col,
            "pearson_r": float(pearson_r),
            "pearson_p": float(pearson_p),
            "spearman_r": float(spearman_r),
            "spearman_p": float(spearman_p),
        })

    if corr_records:
        corr_df = pl.DataFrame(corr_records)
        corr_df = corr_df.with_columns(
            pl.col("pearson_r").abs().alias("abs_pearson")
        ).sort("abs_pearson", descending=True).drop("abs_pearson")
        print("\nTop correlations (by |Pearson r|):")
        print(corr_df.head(30))
        corr_path = MODEL_DIR / "feature_label_correlations.csv"
        corr_df.write_csv(corr_path)
        print(f"💾 Correlation summary saved to: {corr_path}")
    else:
        print("No valid numeric feature-label correlations could be computed.")

    # Categorical Label Analyses
    if is_label_categorical:
        print("\n=== 6.2 Chi-square & Cramér's V (categorical features vs label) ===")
        cat_cols = [
            c for c, dt in zip(X_train_imputed.columns, X_train_imputed.dtypes)
            if dt in {pl.Utf8, pl.Categorical}
        ]
        chi_records: List[Dict[str, Any]] = []
        uniq_labels, label_codes = np.unique(label_series, return_inverse=True)

        for col in cat_cols:
            x_vals = X_train_imputed.select(pl.col(col)).to_series().to_list()
            x_vals = np.array(x_vals, dtype=object)
            uniq_x, x_codes = np.unique(x_vals, return_inverse=True)
            if len(uniq_x) < 2 or len(uniq_labels) < 2:
                continue

            table = np.zeros((len(uniq_x), len(uniq_labels)), dtype=int)
            for i in range(len(x_codes)):
                table[x_codes[i], label_codes[i]] += 1

            try:
                chi2, p, dof, expected = stats.chi2_contingency(table)
                n = table.sum()
                r, k = table.shape
                if min(r, k) > 1:
                    cramers_v = np.sqrt(chi2 / (n * (min(r, k) - 1)))
                else:
                    cramers_v = np.nan
            except Exception:
                continue

            chi_records.append({
                "feature": col,
                "chi2": float(chi2),
                "p_value": float(p),
                "dof": int(dof),
                "cramers_v": float(cramers_v),
                "n": int(n),
            })

        if chi_records:
            chi_df = pl.DataFrame(chi_records).sort("cramers_v", descending=True)
            print("\nTop categorical features by Cramér's V:")
            print(chi_df.head(30))
            chi_path = MODEL_DIR / "categorical_chi2_cramersv.csv"
            chi_df.write_csv(chi_path)
            print(f"💾 Chi-square / Cramér's V summary saved to: {chi_path}")
        else:
            print("No suitable categorical features for chi-square / Cramér's V.")

    else:
        print("\nSkipping chi-square / Cramér's V: label not treated as categorical.")

    # ANOVA for Numeric Features
    if is_label_categorical:
        print("\n=== 6.3 One-way ANOVA (numeric features across label groups) ===")
        anova_records: List[Dict[str, Any]] = []
        uniq_labels = np.unique(label_series)
        # Suppress ConstantInputWarning from scipy.stats.f_oneway
        import warnings
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', category=RuntimeWarning)
            warnings.filterwarnings('ignore', message='(?i).*constant.*')
            warnings.filterwarnings('ignore', message='(?i).*F statistic.*')
            for col in numeric_feature_cols:
                x = X_train_imputed.select(pl.col(col)).to_series().to_numpy().astype(float)
                groups = []
                for level in uniq_labels:
                    mask = (label_series == level) & ~np.isnan(x)
                    vals = x[mask]
                    # Filter out constant groups (variance = 0) to avoid warnings
                    if vals.size > 1 and np.var(vals) > 1e-10:
                        groups.append(vals)
                if len(groups) <= 1:
                    continue

                try:
                    F, p = stats.f_oneway(*groups)
                    # Check for valid results (not inf or nan)
                    if np.isfinite(F) and np.isfinite(p):
                        anova_records.append({"feature": col, "F": float(F), "p_value": float(p)})
                except (ValueError, RuntimeWarning):
                    continue

        if anova_records:
            anova_df = pl.DataFrame(anova_records).sort("p_value")
            print("\nTop features by ANOVA p-value:")
            print(anova_df.head(30))
            anova_path = MODEL_DIR / "anova_numeric_features_vs_label.csv"
            anova_df.write_csv(anova_path)
            print(f"💾 ANOVA summary saved to: {anova_path}")
        else:
            print("No valid ANOVA comparisons could be computed.")
    else:
        print("\nSkipping ANOVA: label not treated as categorical.")

    # Tukey's HSD Post-hoc
    if is_label_categorical and HAS_STATSMODELS:
        try:
            # statsmodels is pandas-based; we only use it to compute Tukey on a few columns
            import pandas as pd  # local import, not used for core data handling
            anova_pdf = anova_df.to_pandas()
            N_TOP_TUKEY = 20
            top_features = anova_pdf.nsmallest(N_TOP_TUKEY, "p_value")["feature"].tolist()
            tukey_records: List[Dict[str, Any]] = []

            for feat in top_features:
                vals = X_train_imputed.select(pl.col(feat)).to_series().to_numpy().astype(float)
                mask = ~np.isnan(vals)
                if mask.sum() < 10:
                    continue
                tukey = pairwise_tukeyhsd(
                    endog=vals[mask],
                    groups=label_series[mask],
                    alpha=0.05,
                )
                summary = tukey.summary()
                header = summary.data[0]
                for row in summary.data[1:]:
                    rec = {h: v for h, v in zip(header, row)}
                    rec["feature"] = feat
                    tukey_records.append(rec)

            if tukey_records:
                tukey_df = pl.DataFrame(tukey_records)
                print("\nSample of Tukey HSD results:")
                print(tukey_df.head(30))
                tukey_path = MODEL_DIR / "tukey_hsd_top_features.csv"
                tukey_df.write_csv(tukey_path)
                print(f"💾 Tukey HSD results saved to: {tukey_path}")
            else:
                print("Tukey's HSD did not produce any results (insufficient data after filtering).")
        except Exception as e:
            print(f"Tukey's HSD step encountered an error and will be skipped: {e}")
    elif is_label_categorical and not HAS_STATSMODELS:
        print("\nTukey's HSD skipped: statsmodels is not installed.")


Label diagnostics:
  dtype        : int64
  # unique vals: 2
  treated as categorical: True

=== 6.1 Pearson & Spearman correlations ===



Top correlations (by |Pearson r|):
shape: (30, 5)
┌──────────────────┬───────────┬───────────┬────────────┬────────────┐
│ feature          ┆ pearson_r ┆ pearson_p ┆ spearman_r ┆ spearman_p │
│ ---              ┆ ---       ┆ ---       ┆ ---        ┆ ---        │
│ str              ┆ f64       ┆ f64       ┆ f64        ┆ f64        │
╞══════════════════╪═══════════╪═══════════╪════════════╪════════════╡
│ has_grants       ┆ 0.325772  ┆ 0.0       ┆ 0.325772   ┆ 0.0        │
│ num_grants       ┆ 0.274682  ┆ 0.0       ┆ 0.328464   ┆ 0.0        │
│ num_institutions ┆ 0.265076  ┆ 0.0       ┆ 0.285334   ┆ 0.0        │
│ num_locations    ┆ 0.25264   ┆ 0.0       ┆ 0.213839   ┆ 0.0        │
│ has_pmid         ┆ 0.24681   ┆ 0.0       ┆ 0.24681    ┆ 0.0        │
│ …                ┆ …         ┆ …         ┆ …          ┆ …          │
│ specter2_dim_504 ┆ -0.155645 ┆ 0.0       ┆ -0.155084  ┆ 0.0        │
│ scibert_dim_440  ┆ -0.155306 ┆ 0.0       ┆ -0.168294  ┆ 0.0        │
│ specter2_dim_533 ┆ -0.15


Top categorical features by Cramér's V:
shape: (3, 6)
┌──────────┬──────────────┬─────────┬────────┬───────────┬────────┐
│ feature  ┆ chi2         ┆ p_value ┆ dof    ┆ cramers_v ┆ n      │
│ ---      ┆ ---          ┆ ---     ┆ ---    ┆ ---       ┆ ---    │
│ str      ┆ f64          ┆ f64     ┆ i64    ┆ f64       ┆ i64    │
╞══════════╪══════════════╪═════════╪════════╪═══════════╪════════╡
│ id       ┆ 960000.0     ┆ 0.49952 ┆ 959999 ┆ 1.0       ┆ 960000 │
│ type     ┆ 47319.050514 ┆ 0.0     ┆ 19     ┆ 0.222015  ┆ 960000 │
│ language ┆ 20000.140598 ┆ 0.0     ┆ 54     ┆ 0.144338  ┆ 960000 │
└──────────┴──────────────┴─────────┴────────┴───────────┴────────┘
💾 Chi-square / Cramér's V summary saved to: /gpfs/accounts/si670f25_class_root/si670f25_class/santoshd/Kaggle_2/data/model_ready/categorical_chi2_cramersv.csv

=== 6.3 One-way ANOVA (numeric features across label groups) ===



Top features by ANOVA p-value:
shape: (30, 3)
┌─────────────────┬───────────────┬─────────┐
│ feature         ┆ F             ┆ p_value │
│ ---             ┆ ---           ┆ ---     │
│ str             ┆ f64           ┆ f64     │
╞═════════════════╪═══════════════╪═════════╡
│ has_abstract    ┆ 7484.918853   ┆ 0.0     │
│ has_doi         ┆ 14211.170527  ┆ 0.0     │
│ has_grants      ┆ 113978.041551 ┆ 0.0     │
│ has_pmid        ┆ 62271.670136  ┆ 0.0     │
│ is_in_doaj      ┆ 22816.719899  ┆ 0.0     │
│ …               ┆ …             ┆ …       │
│ scibert_dim_108 ┆ 18289.870082  ┆ 0.0     │
│ scibert_dim_109 ┆ 7816.826237   ┆ 0.0     │
│ scibert_dim_111 ┆ 5154.4229     ┆ 0.0     │
│ scibert_dim_113 ┆ 8148.418707   ┆ 0.0     │
│ scibert_dim_114 ┆ 6272.114133   ┆ 0.0     │
└─────────────────┴───────────────┴─────────┘
💾 ANOVA summary saved to: /gpfs/accounts/si670f25_class_root/si670f25_class/santoshd/Kaggle_2/data/model_ready/anova_numeric_features_vs_label.csv



Sample of Tukey HSD results:
shape: (20, 8)
┌────────┬────────┬──────────┬───────┬─────────┬─────────┬────────┬──────────────────┐
│ group1 ┆ group2 ┆ meandiff ┆ p-adj ┆ lower   ┆ upper   ┆ reject ┆ feature          │
│ ---    ┆ ---    ┆ ---      ┆ ---   ┆ ---     ┆ ---     ┆ ---    ┆ ---              │
│ i64    ┆ i64    ┆ f64      ┆ f64   ┆ f64     ┆ f64     ┆ f64    ┆ str              │
╞════════╪════════╪══════════╪═══════╪═════════╪═════════╪════════╪══════════════════╡
│ 0      ┆ 1      ┆ 0.1732   ┆ 0.0   ┆ 0.1693  ┆ 0.1771  ┆ 1.0    ┆ has_abstract     │
│ 0      ┆ 1      ┆ 0.1895   ┆ 0.0   ┆ 0.1864  ┆ 0.1926  ┆ 1.0    ┆ has_doi          │
│ 0      ┆ 1      ┆ 0.346    ┆ 0.0   ┆ 0.344   ┆ 0.348   ┆ 1.0    ┆ has_grants       │
│ 0      ┆ 1      ┆ 0.2301   ┆ 0.0   ┆ 0.2283  ┆ 0.2319  ┆ 1.0    ┆ has_pmid         │
│ 0      ┆ 1      ┆ 0.1611   ┆ 0.0   ┆ 0.159   ┆ 0.1631  ┆ 1.0    ┆ is_in_doaj       │
│ …      ┆ …      ┆ …        ┆ …     ┆ …       ┆ …       ┆ …      ┆ …                

## 7. Prepare model-ready feature sets (keep all non-embedding features)


In [7]:
# Keep ALL non-embedding features (no reduction)
# This preserves signals that may be sparse in the sample but informative in the full dataset
# Scalable approach: works with any dataset size using Polars operations

embedding_cols_current: List[str] = []
for m in embedding_models:
    prefix = f"{m}_"
    embedding_cols_current.extend([c for c in X_train_imputed.columns if c.startswith(prefix)])
embedding_cols_current = sorted(set(embedding_cols_current))

# Get all non-embedding features (numeric + categorical)
non_emb_numeric_cols = [
    c for c, dt in zip(X_train_imputed.columns, X_train_imputed.dtypes)
    if dt in {
        pl.Int8, pl.Int16, pl.Int32, pl.Int64,
        pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64,
        pl.Float32, pl.Float64
    }
    and c not in embedding_cols_current 
    and c not in id_cols 
    and c != "label"
]

categorical_cols_current = [
    c for c, dt in zip(X_train_imputed.columns, X_train_imputed.dtypes)
    if dt in {pl.Utf8, pl.Categorical}
    and c not in id_cols
]

print("\nFeature summary:")
print(f"  Embedding features (kept): {len(embedding_cols_current)}")
print(f"  Non-embedding numeric features (kept): {len(non_emb_numeric_cols)}")
print(f"  Categorical features (kept): {len(categorical_cols_current)}")
print(f"  ID columns: {len(id_cols)}")
print(f"\n  Total non-embedding features kept: {len(non_emb_numeric_cols) + len(categorical_cols_current)}")

# Prepare model-ready feature sets (all features)
model_ready_feature_cols = (
    id_cols
    + embedding_cols_current
    + non_emb_numeric_cols
    + categorical_cols_current
)

# Ensure column order and existence
seen: set[str] = set()
ordered_model_ready_cols: List[str] = []
for c in model_ready_feature_cols:
    if c not in seen and c in X_train_imputed.columns:
        ordered_model_ready_cols.append(c)
        seen.add(c)

print(f"\nTotal columns in model-ready feature set (excluding label): {len(ordered_model_ready_cols)}")

# Create model-ready datasets (all features, scalable with Polars)
X_train_model_ready = X_train_imputed.select(ordered_model_ready_cols)
X_val_model_ready   = X_val_imputed.select(ordered_model_ready_cols)
X_test_model_ready  = X_test_imputed.select(ordered_model_ready_cols)

print("\nShapes of model-ready feature matrices:")
print(f"  X_train_model_ready: {X_train_model_ready.shape}")
print(f"  X_val_model_ready:   {X_val_model_ready.shape}")
print(f"  X_test_model_ready:  {X_test_model_ready.shape}")



Feature summary:
  Embedding features (kept): 1920
  Non-embedding numeric features (kept): 54
  Categorical features (kept): 2
  ID columns: 1

  Total non-embedding features kept: 56



Total columns in model-ready feature set (excluding label): 1977

Shapes of model-ready feature matrices:
  X_train_model_ready: (960000, 1977)
  X_val_model_ready:   (120000, 1977)
  X_test_model_ready:  (120000, 1977)


## 8. Save model-ready train / validation / test parquet files


In [8]:
# Prepare final model-ready datasets with labels (scalable with Polars)
train_model_ready = X_train_model_ready
val_model_ready   = X_val_model_ready
test_model_ready  = X_test_model_ready

if y_train is not None:
    train_model_ready = train_model_ready.with_columns(pl.Series("label", y_train))
if y_val is not None and y_val.shape[0] == val_model_ready.height:
    val_model_ready = val_model_ready.with_columns(pl.Series("label", y_val))

print("\nModel-ready shapes:")
print(f"  train_model_ready: {train_model_ready.shape}")
print(f"  val_model_ready:   {val_model_ready.shape}")
print(f"  test_model_ready:  {test_model_ready.shape}")

# Save model-ready parquet files (scalable - Polars handles large datasets efficiently)
train_path = MODEL_DIR / "train_model_ready.parquet"
val_path   = MODEL_DIR / "val_model_ready.parquet"
test_path  = MODEL_DIR / "test_model_ready.parquet"

train_model_ready.write_parquet(train_path)
val_model_ready.write_parquet(val_path)
test_model_ready.write_parquet(test_path)

print("\n💾 Saved model-ready parquet files:")
print(f"  {train_path}")
print(f"  {val_path}")
print(f"  {test_path}")



Model-ready shapes:
  train_model_ready: (960000, 1978)
  val_model_ready:   (120000, 1978)
  test_model_ready:  (120000, 1977)



💾 Saved model-ready parquet files:
  /gpfs/accounts/si670f25_class_root/si670f25_class/santoshd/Kaggle_2/data/model_ready/train_model_ready.parquet
  /gpfs/accounts/si670f25_class_root/si670f25_class/santoshd/Kaggle_2/data/model_ready/val_model_ready.parquet
  /gpfs/accounts/si670f25_class_root/si670f25_class/santoshd/Kaggle_2/data/model_ready/test_model_ready.parquet


## 9. Recap

This notebook:

- Loads the base features from `X_train.parquet`, `X_val.parquet`, `X_test.parquet`
- Discovers and merges all available embedding parquets via `id`
- Performs train-centric imputation with polars
- Replaces `is_oa` with `is_not_oa` (derived feature: 1 - is_oa) for better signal
- Computes Pearson / Spearman, chi-square / Cramér's V, ANOVA, optional Tukey HSD
- Keeps **all non-embedding features** (no reduction) to preserve signals that may be sparse in sample but informative in full dataset
- Writes model-ready splits under `data/model_ready`

All dataframe logic is handled with **polars** (scalable for large datasets); pandas appears only in a small local scope if you enable Tukey HSD via statsmodels.
